---
Title: Introduction To Bitmap File Formats.
Date: 2024-03-01 12:00
Lang: zh-cn
Category: FileFormats
---

# 介绍

Windows 的图形设备接口(GDI)，在 Windows 1.0 开始便支持位图。但此时的位图并不是此文介绍的BMP图像，而是"GDI位图对象"，也被称为"设备相关位图"`(Device-dependent bitmap, DDB)`。

到了20世纪80年代中期，微软与IBM合作开发的操作系统 `OS/2 1.1` 版本中定义了 `DIB` 格式，这就是早期 `BMP` 图像的起源。`OS/2 1.1` 于 1988 年发布，是第一个拥有像 Windows 一样的图形用户界面`(Presentation Manager, PM)`的 `OS/2` 版本。PM 包含一套图形编程接口，其中定义了 `BMP` V1版本的位图格式。

> 参考文档:
- [微软开发文档--位图存储](https://learn.microsoft.com/en-us/windows/win32/gdi/bitmap-storage)


## BMP文件存储结构

BMP图像在磁盘存储的比特分为几个部分: `BITMAPFILEHEADER`, `BITMAPINFOHEADER`(扩展版本`BITMAPV4HEADER`/`BITMAPV5HEADER`) 结构组成位图信息头部结构，以及跟随之后的可选 `RGBQUAD` 结构数组(也称颜色表)和实际的图像数据/索引数组。

- `file_header/BITMAPFILEHEADER(14 bytes)`
    * `bfType(2 bytes)`: 文件类型，必须为 `BM(0x424d)`
    * `bfSize(4 bytes)`: 文件大小(bytes)
    * `bfReserved1(2 bytes)`: 保留字段，必须为0
    * `bfReserved2(2 bytes)`: 保留字段，必须为0
    * `bfOffBits(4 bytes)`: 图像数据偏移量(bytes)，一般为54
- `bmp_info_header/BITMAPINFOHEADER(40 bytes)`
    * `biSize(4bytes)`: 信息头大小(bytes)
    * `biWidth(4bytes)`: 图像宽度(像素)
    * `biHeight(4bytes)`: 图像高度(像素)
    * `biPlanes(2bytes)`: 颜色平面数，必须为1
    * `biBitCount(2bytes)`: 每像素位数(1、4、8、16、24或32)
    * `biCompression(4bytes)`: 压缩类型(0: 无压缩，1: RLE-8，2: RLE-4，3: BITFIELDS)
    * `biSizeImage(4bytes)`: 图像数据大小(bytes)
    * `biXPelsPerMeter(4bytes)`: 水平分辨率(像素/米)
    * `biYPelsPerMeter(4bytes)`: 垂直分辨率(像素/米)
    * `biClrUsed(4bytes)`: 颜色表中使用的颜色数
    * `biClrImportant(4bytes)`: 重要颜色数
- `颜色表(optional)`
    * **注意** 如果biBitCount小于16，则颜色表是必须的
    * 颜色表中的每个颜色由一个RGBQUAD结构表示
- `图像数据`
    * 数据按行存储，行尾可能有填充字节以使行长度为4的倍数
    * **注意** BMP图像格式中，像素数据以"自下而上"，"从左至右" 顺序存储

**注意** BMP文件格式有多种变体，包括OS/2 BMP、Windows BMP和OS X BMP等。这些变体之间的差异主要在于文件头和信息头的格式

**注意** 如果BMP的 `biWidth*biHeight*biBitCount/8 != biSizeImage`, 一般是将每行的像素存储字节扩大为4的倍数(即4字节对齐), 计算公式如: 

$$ RowLength = 4 * \lfloor \frac{biWidth*biBitCount + 31}{32} \rfloor $$
$$ biSizeImage = RowLength * biHeight $$

## BMP压缩类型

- 0: 无压缩(BI_RGB)
- 1: RLE8 压缩(8 位位图)
- 2: RLE4 压缩(4 位位图)
- 3: BITFIELDS 压缩(16 位或 32 位位图)


## BMP 版本

目前为止, `BMP` 共有V1/V2/V3/V4/V5五个版本, 一般通过 `bmp_info_header` 的 `biSize` 字段区分

> 几种 `BMP` 版本的 `BITMAPINFOHEADER` 结构如下:

```c
// =================================================
// OS/2 V1
// GDI 支持: OS/2 及 Windows 3.0 以上的 Windows 版本
// =================================================
typedef struct tagBITMAPCOREHEADER { // 12 字节
  DWORD bcSize;
  WORD  bcWidth;
  WORD  bcHeight;
  WORD  bcPlanes;
  WORD  bcBitCount;
} BITMAPCOREHEADER, FAR *LPBITMAPCOREHEADER, *PBITMAPCOREHEADER;

// =================================================
// OS/2 V2
// =================================================
typedef struct _BITMAPFILEHEADER2 {
    ...
} BITMAPFILEHEADER2;


// =================================================
// Windows V3
// GDI 支持: Windows 3.0 以上的Windows版本
// =================================================
typedef struct tagBITMAPINFOHEADER { // 40 字节
  DWORD  biSize;
  LONG   biWidth;
  LONG   biHeight;
  WORD   biPlanes;
  WORD   biBitCount;
  DWORD  biCompression;
  DWORD  biSizeImage;
  LONG   biXPelsPerMeter;
  LONG   biYPelsPerMeter;
  DWORD  biClrUsed;
  DWORD  biClrImportant;
} BITMAPINFOHEADER, FAR *LPBITMAPINFOHEADER, *PBITMAPINFOHEADER;

// =================================================
// Windows V4
// GDI 支持: Windows 95/NT 4 以上的 Windows 版本
// =================================================
typedef struct { // 108 字节
  DWORD        bV4Size;
  LONG         bV4Width;
  LONG         bV4Height;
  WORD         bV4Planes;
  WORD         bV4BitCount;
  DWORD        bV4V4Compression;
  DWORD        bV4SizeImage;
  LONG         bV4XPelsPerMeter;
  LONG         bV4YPelsPerMeter;
  DWORD        bV4ClrUsed;
  DWORD        bV4ClrImportant;
  DWORD        bV4RedMask;   // 红色遮罩
  DWORD        bV4GreenMask; // 绿色遮罩
  DWORD        bV4BlueMask;  // 蓝色遮罩
  DWORD        bV4AlphaMask; // 阿尔法遮罩
  DWORD        bV4CSType;    // 色彩空间类型
  CIEXYZTRIPLE bV4Endpoints; // XYZ值
  DWORD        bV4GammaRed;  // 红色伽马值
  DWORD        bV4GammaGreen;// 绿色伽马值
  DWORD        bV4GammaBlue; // 蓝色伽马值
} BITMAPV4HEADER, FAR *LPBITMAPV4HEADER, *PBITMAPV4HEADER;

// =================================================
// Windows V5
// GDI 支持: Windows 98/2000 及其新版本
// =================================================
typedef struct { // 124 字节
  DWORD        bV5Size;
  LONG         bV5Width;
  LONG         bV5Height;
  WORD         bV5Planes;
  WORD         bV5BitCount;
  DWORD        bV5Compression;
  DWORD        bV5SizeImage;
  LONG         bV5XPelsPerMeter;
  LONG         bV5YPelsPerMeter;
  DWORD        bV5ClrUsed;
  DWORD        bV5ClrImportant;
  DWORD        bV5RedMask;
  DWORD        bV5GreenMask;
  DWORD        bV5BlueMask;
  DWORD        bV5AlphaMask;
  DWORD        bV5CSType;
  CIEXYZTRIPLE bV5Endpoints;
  DWORD        bV5GammaRed;
  DWORD        bV5GammaGreen;
  DWORD        bV5GammaBlue;
  DWORD        bV5Intent;      // 渲染意图
  DWORD        bV5ProfileData; // 颜色配置数据或文件名(偏移量)
  DWORD        bV5ProfileSize; // 内嵌数据或文件名的大小
  DWORD        bV5Reserved;    // 保留
} BITMAPV5HEADER, FAR *LPBITMAPV5HEADER, *PBITMAPV5HEADER; 
```

**注意** `BMP` 的V2版本在网上信息很小, 这里不确定, 所以没有列出

## BMP操作相关工具

- greenfish图标编辑器, **注意: 1.4版本的位深度转换无效,请使用更高版本(如4.4)**
- ImageMagick开源的图像处理工具

> 资源链接:
[greenfish软件下载页](https://greenfishsoftware.org/gfie.php)